In [34]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from tqdm import tqdm

In [2]:
extracted_coefficients_directory_Bzeta = '../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/Bzeta/'

In [32]:
def find_closest_element(y: float, arr: np.ndarray):
    index = np.searchsorted(arr,y)
    if index >= 1 and index < len(arr):
        res = [arr[index - 1], arr[index]]
    elif index == len(arr) :
        return np.array(index - 1)
    else:
        return index

    if res[0] == res[1]:
        return np.array(index - 1)
    else:
        diff_pre = np.abs(y-res[0])
        diff_aft = np.abs(y-res[1])
        if diff_pre == diff_aft:
            return np.array(index - 1)
        else:
            return index - 1 if diff_pre < diff_aft else index

In [4]:
density_path = '../../data/commaai/density/gaussian_density_filtered.csv'
density = pd.read_csv(density_path)

In [5]:
labels = np.load(str(extracted_coefficients_directory_Bzeta + 'labels_val.npy'))
B_zetas = np.load(str(extracted_coefficients_directory_Bzeta + 'B_zeta_val.npy'))
tr_labels = np.load(str(extracted_coefficients_directory_Bzeta + 'tr_labels_val.npy'))

In [6]:
#beta_dnn = np.genfromtxt('../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/beta/beta.csv', delimiter = ',')
beta_mcmc = np.mean(np.genfromtxt('../../data/commaai/mcmc/filtered_gaussian_resampled/Ridge/betas.csv', delimiter = ',')[10000:,:], axis = 0)
beta_va = np.mean(np.genfromtxt('../../data/commaai/va/filtered_gaussian_resampled/Ridge/mu_t_va.csv', delimiter = ',')[20000:,0:10], axis = 0)


In [7]:
# transformed
pred_mcmc = pd.DataFrame({'pred_z':  [item for sublist in B_zetas.dot(beta_mcmc) for item in sublist]})
pred_va = pd.DataFrame({'pred_z': [item for sublist in  B_zetas.dot(beta_va)for item in sublist]})
mae__z_mcmc = np.mean(abs(pred_mcmc['pred_z'] - tr_labels))
mae_z_va = np.mean(abs(pred_va['pred_z'] - tr_labels))
mse_z_mcmc = np.mean(abs(pred_mcmc['pred_z'] - tr_labels)**2)
mse_z_va = np.mean(abs(pred_va['pred_z'] - tr_labels)**2)
# transformed back

In [35]:
tqdm.pandas()
pred_mcmc['pred_y'] = pred_mcmc['pred_z'].progress_apply(lambda x: norm.cdf(density.loc[find_closest_element(x, density['cdf']), 'axes']))
pred_va['pred_y'] = pred_va['pred_z'].progress_apply(lambda x: norm.cdf(density.loc[find_closest_element(x, density['cdf']), 'axes']))


/home/hofmancl/.local/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 10472/10472 [00:05<00:00, 1855.83it/s]


In [45]:
mae_y_mcmc = np.mean(abs(pred_mcmc['pred_y'] - tr_labels))
mae_y_va = np.mean(abs(pred_va['pred_y'] - tr_labels))
accuracy_mcmc = np.mean(abs(pred_mcmc['pred_y'] - tr_labels) <= 6)
accuracy_va = np.mean(abs(pred_va['pred_y'] - tr_labels) <= 6)
mse_y_mcmc = np.mean(abs(pred_mcmc['pred_y'] - tr_labels)**2)
mse_y_va = np.mean(abs(pred_va['pred_y'] - tr_labels)**2)
print('mean absolute error MCMC: ' + str(mae_y_mcmc) +
     '\nmean absolute error VA: ' + str(mae_y_va) +
     '\naccuracy MCMC ' + str(accuracy_mcmc) +
     '\naccuracy VA ' + str(accuracy_va) +
     '\nmean squared error MCMC: ' + str(mse_y_mcmc) +
     '\nmean squared error VA: ' + str(mse_y_va) )

mean absolute error MCMC: 0.6748485246934615
mean absolute error VA: 0.6748975096475874
accuracy MCMC 1.0
accuracy VA 1.0
mean squared error MCMC: 0.7501234890574127
mean squared error VA: 0.7502340404841749


In [49]:
np.sqrt(0.7502340404841749)

0.866160516581179

In [29]:
find_closest_element(1.5, density['cdf'])

array(65535)

In [19]:
density['cdf']

0        1.827306e-09
1        3.652270e-09
2        5.475983e-09
3        7.299531e-09
4        9.124003e-09
             ...     
65531    1.000000e+00
65532    1.000000e+00
65533    1.000000e+00
65534    1.000000e+00
65535    1.000000e+00
Name: cdf, Length: 65536, dtype: float64